# Generating masks

This file genertes boolean masks to identify panels and faults in the imagens generated by the Unity 3D simulation. The masks for faults are generated separated from the masks for the panels.

There as masks separated in files (on mask each each) and also a file that contains all the masks for each kind. Files are saved in HDF5 format.

## Modules

- `os` - to read files
- `numpy` -  math functions and matrices
- `shapely` - define contours to panel's masks
- `PIL` - image reading
- `h5py`- HDF5 files, good for matrices

In [85]:
import os
import numpy as np
from shapely.geometry import Point,Polygon
from PIL import Image
import h5py as h5

## Masks for panels


There is a `csv` file with the screeb coordinates for the four corners of the panels for every panel in every image.

The columns of the `csv` file are the following:
1. Image id
2. Panel id
3. x coordinate of first corner
4. y coordinate of first corner
5. x coordinate of second corner
6. y coordinate of second corner
7. x coordinate of third corner
8. y coordinate of third corner
9. x coordinate of fourth corner
10. y coordinate of fourth corner

The file is read and the numbers ar taken as points in a plne. Then the `shapely`module is used when creating a convex polygon that represents the panel. Screen coordinates inside tha polygon are marked `true` and outside, `false`.


In [50]:
# Using the matrix with all the lines 
# of the original csv file,
# only the line that refer to a panel 
# in a image are taken.
def linesForPanelImage(panel,image,allLinesMatrix):
    pc = allLinesMatrix
    
    linesImg = pc[:,0]==image # selects the image
    linesPi = pc[:,1]==panel # selects the panel
    lines = linesPi * linesImg # logic intersection
    
    return pc[lines,:]

In [51]:
# creates a list of plane points that are the corners 
# of the panels in a image.
# Selects the columns of the matrix with the 
# x and y coordinates.
def cornersOfPanel(dataMatrix):
    
    # collects the points in the data matrix
    c = []
    for i in range(npanels):
        c.append(dataMatrix[:,(2+i*3):(4+i*3)].squeeze())

    # assemble list of points 
    # as objects of approriate class
    # of shapely module
    p = []
    for pi in c:
        p.append(Point(pi[0],pi[1]))

    # creates a polygon with the corners and
    # applies convex hull to determine correct 
    # order of points
    pg = Polygon(p)
    pg = pg.convex_hull
    
    return pg

In [52]:
# iterates over all imagens and all panels
# collecting the polygons that
# define the panels
def cornersOfAllPanels(pointsMatrix,ninmages,npanels=4):
    for img in range(nimages):
        pg = []
        for pi in range(npanels):
            panelData = linesForPanelImage(pi,img,pointsMatrix)
            pg.append(cornersOfPanel(panelData))
        
    return pg

In [53]:
# uses thes polygons in the image coordinates
# to generate a boolean mask 
def generateMask(pg,width,height):
    
    mask = np.zeros([width,height],dtype=bool)        
    
    for i in range(width):
        for j in range(height):
            # every pixel is taken as point
            p = Point(i,j)
            for k in range(npanels):
                # function 'within' in 'shapely'
                # tests if a point is
                # inside a polygon
                if p.within(pg[k]):
                    mask[i,j] = True
                    
    return mask

### Main procedure

Generate masks for panels.

In [71]:
# folder with images and  csv files
folder = '../../data'
files = os.listdir(folder)
panelsCorners = folder+'/panelsScreen.csv'

# number of panels
npanels = 4

# select a image just to read dimensions
files = os.listdir(folder)
file = files[10]
img = Image.open(folder+'/'+file)
width = img.width
height = img.height

# folder where to store masks
masksfolder = '../../Detection NN/masks/'

# file with all masks
ft = h5.File(masksfolder+'masks.h5','w')
# read file
pc = np.loadtxt(panelsCorners,delimiter=',')

for file in files:
    # select image files saved as 'screen_id.png'
    if 'screen_' in file:   
        # polygons that surround panels in images
        pg = cornersOfAllPanels(pc,width,height,nimages,npanels)
        # masks
        mask = generateMask(pg,width,height)
        
        # add a maks to the file with all masks
        ft.create_dataset(file[:-4],mask.shape,data=mask)
        
        # create, save data and close a HDF5 file
        # with the mask for a single image
        fi = h5.File(masksfolder+file[:-4]+'.h5','w')
        fi.create_dataset('data',mask.shape,data=mask)
        fi.close()

# close file with all masks
ft.close()

## Masks for faults

The masks for the faults are generated as circles around the faults' center. They are not dimensionless, since the are elipses whose area is proportional to the number called severity. Details for their generation are in the Unity simulation that creates the images.

The `csv` files for the faults have columns as follows (each line contains the data for a fault):
1. Image id
2. Panel id
3. x screen coordinate
4. y screen coordinate
5. severity

The procedure in this case is simpler and thus not divided into several methods.

In [ ]:
# folder with images and  csv files
folder = '../../data'

# select a image just to read dimensions
files = os.listdir(folder)
file = files[10]
img = Image.open(folder+'/'+file)
width = img.width
height = img.height

# folder where to stores the masks
masksfolder = '../../masks/'

# file for the masks of all images
ft = h5.File(masksfolder+'faults_masks.h5','w')

# csv file with faults locations and severity
faultsCorners = folder+'/faultsScreen.csv'
fc = np.loadtxt(faultsCorners,delimiter=',')

# creating a pair of matrices containing
# x and y coordinates of pixels
# they are used in the calculation
# of a circle around a pixel
x = np.arange(width)
y = np.arange(height)
Y,X = np.meshgrid(y,x)

# iterating over the image files
for file in files:
    if 'screen_' in file:   
        
        mask = np.zeros([width,height],dtype=bool) 
        # identifies image id as in the file name
        # screen_id.png
        iimg = int(file.split('.')[0].split('_')[1])
        # faults of a particular image
        faultsIndex = fc[:,0]==iimg
        position = fc[faultsIndex,2:4]
        severity = fc[faultsIndex,-1]
        
        # for every fault, calculated a circle
        # around it proportional to the severity
        for pos,sev in zip(position,severity):
            
            # the circle around the fault
            # must not escape the borders
            # of the image
            Xd = np.clip(X-pos[0],0,width-1)
            Yd = np.clip(Y-pos[1],0,height-1)
            # circle inequation
            insideFault = Xd*Xd+Yd*Yd<10*sev*sev
            
            # logical 'or' to include another
            # fault in the mask for current image
            mask = np.logical_or(mask, insideFault)

        # add current mask to the file
        ft.create_dataset(file[:-4],mask.shape,data=mask)
        
        # create, stores and closes
        # mask for current image in a separate file
        fi = h5.File(masksfolder+'fm_'+file[:-4]+'.h5','w')
        fi.create_dataset('data',mask.shape,data=mask)
        fi.close()

# close file for all images
ft.close()
        